<a href="https://colab.research.google.com/github/cheul0518/Ccompetitions/blob/main/HuBMAP_HPA/DataSizeConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HuBMAP HPA data converter

In [ ]:
# Mount your Google Drive. You can permanently mount yours if you've allowed "Mount Drive" on Files section (left panel)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ~/ : root/
#!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/DeepLearning/HubMAP/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c hubmap-organ-segmentation
!unzip /content/hubmap-organ-segmentation.zip -d /content/hubmap-organ-segmentation
!rm /content/hubmap-organ-segmentation.zip

Archive:  /content/hubmap-organ-segmentation.zip
  inflating: /content/hubmap-organ-segmentation/sample_submission.csv  
  inflating: /content/hubmap-organ-segmentation/test.csv  
  inflating: /content/hubmap-organ-segmentation/test_images/10078.tiff  
  inflating: /content/hubmap-organ-segmentation/train.csv  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10044.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10274.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10392.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10488.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10610.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10611.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10651.json  
  inflating: /content/hubmap-organ-segmentation/train_annotations/10666.json  
  inflating: /content/hubmap-organ-segmentation/train_a

In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 7.9 MB/s 


##Basic concept idea of resizing data is:

  - (img_size + padding) / (tile_size * resize_ratio) = # of patches
      - Adding padding makes (img_size + padding) % (tile size * resize_ratio) = 0

  - Patch.resize(tile_size, interpolation=inter_area)
      - Resize_ratio determines the patch area. If resize_ratio = 3, for example, then the area is 9 (width * height in a patch). So the resized patch (=tile size) is 9 times smaller and less precise than the original patch (=tile_size * resize_ratio)

In [ ]:
import gc
import os
import cv2
import zipfile
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tiff
from PIL import Image
from tqdm.notebook import tqdm
from rasterio.windows import Window
from torch.utils.data import Dataset

TRAIN_IMG = 'train_img.zip'
TRAIN_MASK = 'train_mask.zip'
TILE_SIZE = 256 # tile size 
RESIZE_RATIO = 4 # if 4, then divide 
IMGS = '/content/hubmap-organ-segmentation/train_images'
RLE = '/content/hubmap-organ-segmentation/train.csv'

In [ ]:
# functions to convert encoding to mask and vice versa
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    # For this code, i is meaningless. But I keep it for later use for multiclass encs
    for i, enc in enumerate(encs):
        if isinstance(enc, float) and np.isnan(enc): 
            continue
        rle_list = enc.split()
        for j in range(len(rle_list)//2):
            start = int(rle_list[2*j]) - 1
            length = int(rle_list[2*j+1])
            img[start:start+length] = 1 + i # i for multi-class encs
    return img.reshape(shape).T

def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1, n+1):
        p = (pixels == i).astype(np.uint8)
        if p.sum() == 0:
            encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])
            runs[1::2] -= runs[::2]
            encs.append(' '.join(map(str, runs)))
            #encs.append(' '.join(str(x) for x in runs)) 

df_rle = pd.read_csv(RLE)[['id', 'rle']].set_index('id')
print(df_rle.head())

                                                     rle
id                                                      
10044  1459676 77 1462675 82 1465674 87 1468673 92 14...
10274  715707 2 718705 8 721703 11 724701 18 727692 3...
10392  1228631 20 1231629 24 1234624 40 1237623 47 12...
10488  3446519 15 3449517 17 3452514 20 3455510 24 34...
10610  478925 68 481909 87 484893 105 487863 154 4908...


In [ ]:
# one of the new images cannot be loaded into 16GB RAM
# use rasterio to load image part by part

sat_th = 40 # saturation threshold
pixel_th = 1000 * (TILE_SIZE // 256) ** 2 # minimum # of pixels

class HuBMAPDataset(Dataset):
      def __init__(self, idx, tile_size=TILE_SIZE, ratio=RESIZE_RATIO, encs=None):
          self.data = rasterio.open(os.path.join(IMGS, str(idx) + '.tiff'), num_threads='all_cpus')
          # type(self.data): <class 'rasterio.io.DatasetReader'>
          # self.data: <open DatasetReader name='/content/hubmap-organ-segmentation/train_images/10044.tiff' mode='r'>

          # self.data.count: # of raster bands
          if self.data.count != 3:
              subdatasets = self.data.subdatasets
              self.layers = []
              if len(subdatasets) > 0:
                  for i, subdataset in enumerate(subdatasets):
                      self.layers.append(rasterio.open(subdataset))
          self.shape = self.data.shape
          self.ratio = ratio
          self.patch = tile_size * ratio
          self.pad0 = (self.patch - self.shape[0] % self.patch) % self.patch
          self.pad1 = (self.patch - self.shape[1] % self.patch) % self.patch
          self.patch_n0 = (self.shape[0] + self.pad0) // self.patch # maximum # of patches in height of image
          self.patch_n1 = (self.shape[1] + self.pad1) // self.patch # maximum # of patches in width of image
          self.mask = enc2mask(encs, (self.shape[1], self.shape[0])) if encs is not None else None

      def __len__(self):
          return self.patch_n0 * self.patch_n1

      def __getitem__(self, idx):
          # n0, n1: index of the patch          
          n0, n1 = idx//self.patch_n0, idx%self.patch_n1
          # x0, y0: coordinates of the top left corner of the patch in the image
          x0, y0 = -self.pad0//2 + n0 * self.patch, -self.pad1//2 + n1 * self.patch 

          # patch coordinates to copy image data, so p00,p10 >= 0, p01,p11 <= image height, image width
          p00, p01 = max(0, x0), min(x0 + self.patch, self.shape[0])
          p10, p11 = max(0, y0), min(y0 + self.patch, self.shape[1])
          img_padded = np.zeros((self.patch, self.patch, 3), np.uint8)
          mask_padded = np.zeros((self.patch, self.patch), np.uint8)

          if self.data.count == 3:
              img_padded[(p00-x0):(p01-x0), (p10-y0):(p11-y0)] = np.moveaxis(self.data.read([1,2,3], 
                                                                           window=Window.from_slices((p00,p01), (p10,p11))), 0, -1)
          else:
              for i, layer in enumerate(self.layers):
                  img_padded[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] = layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))
          if self.mask is not None:
              mask_padded[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = self.mask[p00:p01, p10:p11]

          if self.ratio != 1:
              img_padded = cv2.resize(img_padded,(self.patch//self.ratio,self.patch//self.ratio),interpolation =cv2.INTER_AREA)
              mask_padded = cv2.resize(mask_padded,(self.patch//self.ratio,self.patch//self.ratio),interpolation=cv2.INTER_NEAREST)
          
          # check for empty images
          hsv = cv2.cvtColor(img_padded, cv2.COLOR_BGR2HSV)
          h, s, v = cv2.split(hsv)

          return img_padded, mask_padded, (-1 if(s>sat_th).sum()<=pixel_th or img_padded.sum()<=pixel_th else idx)

x_tot,x2_tot = [],[]
with zipfile.ZipFile(TRAIN_IMG,'w') as img_out, zipfile.ZipFile(TRAIN_MASK,'w') as mask_out:
      for index, encs in tqdm(df_rle.iterrows(), total=len(df_rle)):
          dataset = HuBMAPDataset(idx=index, encs=encs)
          for i in range(len(dataset)):
              img, mask, idx = dataset[i]
              # Less information than threshold, drop
              if idx < 0:
                  continue
              x_tot.append((img/255.0).reshape(-1,3).mean(axis=0))
              x2_tot.append(((img/255.0)**2).reshape(-1,3).mean(axis=0))
              img = cv2.imencode('.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))[1]
              img_out.writestr(f'{index}_{idx:04d}.png', img)
              mask = cv2.imencode('.png', mask)[1]
              mask_out.writestr(f'{index}_{idx:04d}.png', mask)

#image stats
img_avr = np.array(x_tot).mean(axis=0)
img_std = np.sqrt(np.array(x2_tot).mean(axis=0)- img_avr**2)
print('mean:', img_avr, ', std:', img_std)

  0%|          | 0/351 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


mean: [0.7720342  0.74582646 0.76392896] , std: [0.24745085 0.26182273 0.25782376]


In [ ]:
columns, rows = 4, 4
idx0 = 20
fig = plt.figure(figsize=(columns*4, rows*4))
with zipfile.ZipFile(TRAIN_IMG, 'r') as img_arch, zipfile.ZipFile(TRAIN_MASK, 'r') as mask_arch:
      fnames = sorted(img_arch.namelist())[8:]
      print(fnames)
      print(sorted(img_arch.namelist()))
      print(fnames[idx0:idx0+16])
      # for i in range(rows):
      #     for j in range(columns):
      #         idx = i + j*columns
      #         img = cv2.imdecode(np.frombuffer(img_arch.read(fnames[idx0+idx]), np.uint8), cv2.IMREAD_COLOR)
      #         img =cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
      #         mask = cv2.imdecode(np.frombuffer(mask_arch.read(fnames[idx0+idx]), np.uint8), cv2.IMREAD_GRAYSCALE)
      #         fig.add_subplot(rows, columns, idx+1)
      #         plt.axis('off')
      #         plt.imshow(Image.fromarray(img))
      #         plt.imshow(Image.fromarray(mask), alpha=0.2)
plt.show()

['10274_0001.png', '10274_0003.png', '10274_0004.png', '10274_0005.png', '10274_0006.png', '10274_0007.png', '10274_0008.png', '10392_0000.png', '10392_0001.png', '10392_0002.png', '10392_0003.png', '10392_0004.png', '10392_0005.png', '10392_0006.png', '10392_0007.png', '10392_0008.png', '10488_0001.png', '10488_0004.png', '10488_0005.png', '10488_0007.png', '10488_0008.png', '10610_0000.png', '10610_0001.png', '10610_0002.png', '10610_0003.png', '10610_0004.png', '10610_0005.png', '10610_0006.png', '10610_0007.png', '10610_0008.png', '10611_0000.png', '10611_0001.png', '10611_0002.png', '10611_0003.png', '10611_0004.png', '10611_0005.png', '10611_0006.png', '10611_0007.png', '10611_0008.png', '10651_0000.png', '10651_0001.png', '10651_0002.png', '10651_0003.png', '10651_0004.png', '10651_0005.png', '10651_0007.png', '10651_0008.png', '10666_0000.png', '10666_0001.png', '10666_0002.png', '10666_0003.png', '10666_0004.png', '10666_0005.png', '10666_0006.png', '10666_0007.png', '10666_00

<Figure size 1152x1152 with 0 Axes>